In [ ]:
#!pip install pyhive
#!yum uninstall cyrus-sasl-devel-2.1.26-23.el7.x86_64
#!pip install thrift==0.9.3
#!pip install thrift==0.10.0
#!pip install thrift-sasl==0.3.0
#!pip install impyla

In [3]:
import os
os.environ['http_proxy'] = "http://172.30.12.56:3128" 
os.environ['https_proxy'] = "https://172.30.12.56:3128"  
!conda install -y pyhive

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pyhive


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cyrus-sasl-2.1.27          |       he38ecfd_0         232 KB  conda-forge
    future-0.18.2              |           py37_0         711 KB  conda-forge
    libntlm-1.4                |    h14c3975_1002          32 KB  conda-forge
    pyhive-0.6.1               |           py37_0          81 KB  defaults
    sasl-0.2.1                 |py37he1b5a44_1001          56 KB  conda-forge
    thrift-0.11.0              |py37he1b5a44_1001         109 KB  conda-forge
    thrift_sasl-0.3.0          |py37h516909a_1001          13 KB  conda-forge
    ----------------

In [9]:
from sqlalchemy import create_engine
from pyhive import hive

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

#engine = create_engine('hive://clspromon-aio01:10000/default')
engine = create_engine('hive://172.30.17.145:10000/default')

In [ ]:
row =  engine.execute("describe table `default`.`tables_stage3_bic_client`")

In [ ]:
row =  engine.execute("select * from `default`.`mdm_locations_nrw_stage1`")

In [ ]:
import pandas as pd
locationsDF = pd.DataFrame(row)
locationsDF.reset_index(drop=True, inplace=True)
locationsDF.index = locationsDF[0]
locationsDF

In [ ]:
def traffic_color(x):
    return {
        'FREE_FLOW': 'green',
        'CONGESTED': 'yellow',
        'HEAVY': 'red',
        'IMPOSSIBLE' : 'black'
    }[x]

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, LayerGroup, FullScreenControl
#print(df.shape)
hagen = (51.36081, 7.47168)
mmap = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.DE, ), ),
    center=hagen,
    zoom=11,
    scroll_wheel_zoom=True
)
layers = []

for index, row in locationsDF.iterrows():
    cm = CircleMarker()
    cm.location = (row[1],  row[2])
    cm.radius = 10
    cm.color = 'black'
    cm.fill_color = 'green'
    cm.fill_opacity = 1.0
    #cm.weight = 1
    layers.append(cm)
  
    
layer_group = LayerGroup(layers=layers)
mmap.add_layer(layer_group)
mmap.add_control(FullScreenControl())

In [ ]:
import pandas as pd
from confluent_kafka import Consumer, KafkaError
from confluent_kafka.avro import AvroConsumer
from confluent_kafka.avro.serializer import SerializerError

In [ ]:
c = AvroConsumer({
    'bootstrap.servers': 'confluent:9092',
    'group.id': 'groupid3',
    'session.timeout.ms': 6000,
    'default.topic.config': {'auto.offset.reset': 'latest'},
    'schema.registry.url': 'http://confluent:8081'})
c.subscribe(['S2_3391002_TRAFFIC_CAR_DEC'])
df = pd.DataFrame([])

In [ ]:
running = True
while running:
    msg = None
    try:
        msg = c.poll(10)
        if msg:
            if not msg.error():
                
                f = pd.Series(msg.value())
                df[f["ID"]] = f  
                #layers[locationsDF.index.get_loc(f["ID"])].fill_color = traffic_color(f["TRAFFICSTATUS"])
                print(f)
                c.commit(msg)
            elif msg.error().code() != KafkaError._PARTITION_EOF:
                print(msg.error())
                running = False
        else:
            print(".")
    except SerializerError as e:
        print("Message deserialization failed for %s: %s" % (msg, e))
        running = False
c.commit()
c.close()

In [ ]:
mmap

In [ ]:
inspector = inspect(engine)
#row = inspector.get_columns('default')
print(inspector.get_table_names())

In [6]:
inspector = inspect(engine)

# Get table information
#print(inspector.get_table_names())

# Get column information
row=inspector.get_columns('tables_stage1_p_meta_complete')

col = [] 
for _row in row:
    print(_row['name'])
    col.append(_row['name'])

col

product
service
__name__
alertname
alertstate
instance
job
severity
code
le
quantile
scrape_job
interval
event
role
name
config
call
endpoint
rule_group
handler
slice
branch
goversion
revision
version
listener_name
dialer_name
reason
area
runtime
vendor
gc
pool
cache
adapterid
adapter
type
enginetype
status
remotegroupid
remoteinstanceid
remotesystemid
queue
env
state
logical_system
prio
port
drive
account
instanceid
queuename


['product',
 'service',
 '__name__',
 'alertname',
 'alertstate',
 'instance',
 'job',
 'severity',
 'code',
 'le',
 'quantile',
 'scrape_job',
 'interval',
 'event',
 'role',
 'name',
 'config',
 'call',
 'endpoint',
 'rule_group',
 'handler',
 'slice',
 'branch',
 'goversion',
 'revision',
 'version',
 'listener_name',
 'dialer_name',
 'reason',
 'area',
 'runtime',
 'vendor',
 'gc',
 'pool',
 'cache',
 'adapterid',
 'adapter',
 'type',
 'enginetype',
 'status',
 'remotegroupid',
 'remoteinstanceid',
 'remotesystemid',
 'queue',
 'env',
 'state',
 'logical_system',
 'prio',
 'port',
 'drive',
 'account',
 'instanceid',
 'queuename']

In [8]:
import pandas as pd
#row =  engine.execute("describe table `default`.`tables_stage1`")
#row =  engine.execute("select * from tables_stage1_p")
row =  engine.execute("select * from  tables_stage1_p_meta_complete")
df = pd.DataFrame(row)
#df.columns = ['a', 'b','c', 'd','e', 'f','g', 'h', 'i']
df.columns = col
df['__name__']

0                                bis_jms_scheduled_count
1                bis_order_execution_time_seconds_bucket
2                         bis_order_sync_execution_total
3                  scrape_samples_post_metric_relabeling
4                                     bis_adapter_status
                              ...                       
26681              prometheus_target_sync_length_seconds
26682           prometheus_target_scrape_pool_sync_total
26683         prometheus_http_response_size_bytes_bucket
26684    prometheus_http_request_duration_seconds_bucket
26685                         go_gc_duration_seconds_sum
Name: __name__, Length: 26686, dtype: object

In [ ]:
import pandas as pd
#row =  engine.execute("describe table `default`.`tables_stage1`")
row =  engine.execute("select * from tables_stage3_bic_client")
df = pd.DataFrame(row)
df.columns = ['a', 'b','c', 'd','e', 'f','g', 'h', 'i']
df

In [ ]:
import pandas_profiling
import numpy as np
df.describe()

In [ ]:
pandas_profiling.ProfileReport(df)